# 01. 데이터 탐색 (Data Exploration)

수집된 Airbnb 데이터의 기본 통계, 분포, 이상치를 확인합니다.

In [ ]:
import sys
sys.path.insert(0, '..')

import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from sqlalchemy import create_engine, text
from models.database import session_scope
from models.schema import Station, Listing, CalendarSnapshot, DailyStat

In [ ]:
# DB 연결
with session_scope() as session:
    stations = pd.read_sql(session.query(Station).statement, session.bind)
    listings = pd.read_sql(session.query(Listing).statement, session.bind)
    daily_stats = pd.read_sql(session.query(DailyStat).statement, session.bind)

print(f'역 수: {len(stations)}')
print(f'숙소 수: {len(listings)}')
print(f'daily_stats 레코드 수: {len(daily_stats)}')

## 1. 숙소 분포

In [ ]:
# 숙소 유형 분포
room_type_counts = listings['room_type'].value_counts().reset_index()
room_type_counts.columns = ['room_type', 'count']
fig = px.pie(room_type_counts, names='room_type', values='count',
             title='숙소 유형 분포')
fig.show()

In [ ]:
# 역별 숙소 수 상위 20개
listing_by_station = (
    listings.groupby('nearest_station_id').size()
    .reset_index(name='count')
    .merge(stations[['id', 'name']], left_on='nearest_station_id', right_on='id')
    .sort_values('count', ascending=False)
    .head(20)
)
fig = px.bar(listing_by_station, x='name', y='count',
             title='역별 숙소 수 (상위 20개)',
             labels={'name': '역명', 'count': '숙소 수'})
fig.show()

## 2. 가격 분포

In [ ]:
# 기본 가격 분포
listings_with_price = listings[listings['base_price'].notna() & (listings['base_price'] > 0)]
print(listings_with_price['base_price'].describe())

fig = px.histogram(listings_with_price, x='base_price', nbins=50,
                   title='숙소 기본 가격 분포 (KRW)',
                   labels={'base_price': '1박 가격 (KRW)'})
fig.show()

In [ ]:
# 유형별 가격 비교
fig = px.box(listings_with_price, x='room_type', y='base_price',
             title='숙소 유형별 가격 분포',
             labels={'room_type': '숙소 유형', 'base_price': '1박 가격 (KRW)'})
fig.show()

## 3. daily_stats 기본 통계

In [ ]:
# room_type=None (전체) 통계만 분석
total_stats = daily_stats[daily_stats['room_type'].isna()].copy()
total_stats['date'] = pd.to_datetime(total_stats['date'])

print('예약률 통계:')
print(total_stats['booking_rate'].describe())
print()
print('평균 일일 가격 통계:')
print(total_stats['avg_daily_price'].describe())

In [ ]:
# 예약률 히스토그램
fig = px.histogram(total_stats, x='booking_rate', nbins=20,
                   title='역별 예약률 분포',
                   labels={'booking_rate': '예약률'})
fig.show()

## 4. 이상치 탐지

In [ ]:
# 이상치 기준: 가격이 IQR * 1.5 초과
Q1 = listings_with_price['base_price'].quantile(0.25)
Q3 = listings_with_price['base_price'].quantile(0.75)
IQR = Q3 - Q1
outlier_threshold = Q3 + 1.5 * IQR

outliers = listings_with_price[listings_with_price['base_price'] > outlier_threshold]
print(f'이상치 숙소 수: {len(outliers)} (전체의 {len(outliers)/len(listings_with_price):.1%})')
print(f'이상치 기준 가격: {outlier_threshold:,.0f} KRW')
outliers[['name', 'room_type', 'base_price']].sort_values('base_price', ascending=False).head(10)